In [1]:
API_KEY = api_key


In [2]:
def API_func(API_KEY, pdf_route, with_qa, with_list, excel_route, main_content = "", question_symbol = 'Q：', answer_symbol = 'A：', 
             list_symbols = ['●','· '], boundary = 150):
    
    '''
    API_KEY: API key
    pdf_route: route to access target pdf (including information needed to be transformed to questions) / 
               需要access pdf的路徑，如果在同一個文件夾底下則直接為pdf名
    with_qa: whether the pdf file consist of any QA pairs (return True if so) --> connected with variables: question_symbol, answer_symbol / 
            pdf裡是否已經有QA了（有QA需要單獨摘出就是True  -->對應variable：question_symbol，answer_symbol
    with_list: whether there are sections in the pdf that needs the be rewritten (return true if so) --> connected with varialbes: main_content，list_symbols，boundary / 
                需不需要重寫段落（有list需要重寫就是True  -->對應variable：main_content，list_symbols，boundary
    main_content: Main content of the article (e.g. target company's name, key information that would help GPT perform) / 
                    需要重寫問題的時候所需要的段落內容（e.g. 中信銀行，八維智能公司，經濟部，等
    question_symbol: How QA sets in the pdf is formatted (e.g. "Q:" or "Question: " or "Question 1:") / 
                    如果withqa=True才會有用。用來決定Q和A是怎麼樣開始的（e.g. “問題：”  “Q：” 都是可能的情況
    answer_symbol: How QA sets in the pdf is formatted (e.g. "A:" or "Answer:" or "Answer 1:") / 
                    同question_symbol
    list_symbols: If there are sections to be rewritten, what kind of symbol should be used to separate each section （e.g. '●'  '· ') / 
                  如果需要重寫段落，要用怎麼樣的符號來切割每一條內容（e.g. '●'  '· '
    boundary: A selecting variable to decided whether a section needs to be rewritten / 
              如果需要重寫段落，段落應該少於多少字
    excel_route: route to output excel / 
                 最終excel要存的地方+名字
    '''
    
    
    #all needed functions
    
    #Substring questions and answers from unprocess_qa / 從unprocess_qa中提取QA
    def find_qa(section, target_dict, question_symbol, answer_symbol): #variable
        q = section.find(question_symbol)      #variable: 原來：'Q：'
        q_next = section[q+2:].find(question_symbol)
        
        a = section.find(answer_symbol)      #variable: 原來：'A：'
        
        if q_next == -1:
            target_dict[section[q+2:a]] = section[a+2:]
            return target_dict
        
        target_dict[section[q+2:a]] = section[a+2:q_next+q+2]
        
        nxt = section[q_next:]
        target_dict = find_qa(nxt[nxt.find(question_symbol):], target_dict, question_symbol, answer_symbol)
        
        return target_dict
    
    
    def withqa(content, question_symbol, answer_symbol):
        withqa = []
        for x in range(len(content)):
            if (len(content[x].split(question_symbol)) != 1) | (len(content[x].split(answer_symbol)) != 1):
                withqa.append(x)
        
        #combine all QA contents / 將所有QA部分和起來變成一個文章
        all_content_q = ""
        for ind in withqa:
            all_content_q += content[ind]       
        
        content_q = {}
        content_q = find_qa(all_content_q, content_q, question_symbol, answer_symbol)
        return withqa, content_q
    
    
    def get_all_poss(list_symbols, x, withoutlist, content):
        for each in list_symbols:
            if len(content[x].split(each)) != 1:
                withoutlist[x] = content[x].split(each)
        return withoutlist
    
    def get_need_rewrite(withoutlist, boundary):
        need_rewrite = []
        for keys in withoutlist.keys():
            temp = True
            for each in withoutlist[keys]:
                if len(each) > boundary:       #Select using boundary whether a section needs to be rewritten / 劃分怎麼樣需要api重寫的variable:預設值150
                    temp = False
                    break
            if temp:
                need_rewrite.append(keys)
        return need_rewrite
    
    
    def form_new_content(content, to_article = [], withoutlist = {}, withqa = [], need_rewrite = [], unprocess_article = {}):
        new_content = []
        for x in range(len(content)):
            target = ""
            if x in to_article:
                for each in withoutlist[x]:
                    target += each
                new_content.append(target)
                continue
            if x in withqa:
                continue
            if x in need_rewrite:
                new_content.append(unprocess_article[x])
                continue
            new_content.append(content[x])
        return new_content
    
    
    def withlist(content):
        withoutlist = {}         #Selection of all sections with a list / 選出所有帶有list的文章段落
        for x in range(len(content)): 
            withoutlist = get_all_poss(list_symbols, x, withoutlist, content)
            
        need_rewrite = get_need_rewrite(withoutlist, boundary)
        to_article = [x for x in set(withoutlist.keys()) if x not in set(need_rewrite)] #select sections with list but doesn't need to be rewritten / 挑選出有list但不需要重寫的部分
        
        unprocess_article = {}
        if len(need_rewrite) != 0:
            for ind in need_rewrite:
                try:
                    completion = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "user", "content": f"接下來這段文字內容是有關於{main_content}的。把這段文字內容的所有重點重新寫成一個文字段落。文字內容：{withoutlist[ind]}"}
                        ],
                        temperature = 0.2
                    )
                    unprocess_article[ind] = completion.choices[0].message['content']     #unprocess_article: to save the new section / unprocess_article儲存重新寫出來的article -》應該不需要process直接丟進content
                except BaseException:
                    unprocess_article[ind] = ""
        
        return to_article, withoutlist, need_rewrite, unprocess_article
    
    
    def get_qa_1by1(section, target_dict): #Substring questions and answers from unprocess_qa (method 1) / 從unprocess_qa中提取QA --》method1
        if section.rfind('注意：') != -1:        #delete notices that sometimes come up in answers / 去掉有時候會出現的注意通告。之後可能需要增加。冒號是重點！！
            section = section[:section.rfind('注意：')]             

        q = section.find('問題：')
        q_next = section[q+3:].find('問題：')
        a = section.find('答案：')

        if q_next == -1:
            target_dict[section[q+3:a]] = section[a+3:]
            return target_dict

        target_dict[section[q+3:a]] = section[a+3:q_next+1]
        nxt = section[q_next:]

        target_dict = get_qa_1by1(nxt[nxt.find('問題：'):], target_dict)
        return target_dict
    
    
    def get_qa_by_list(section, target_dict):          #method2
        q_start = section.find('問題：')      #find the start of all questions / 找到question list的開頭
        a_start = section.find('答案：')      #find the start of all answers / 找到answer list的開頭
        q_section = section[q_start:a_start]
        a_section = section[a_start:]

        count = 1
        while True:
            q = q_section.find(f"{count}. ")        #find index of questions / 找數字：e.g. 1. 2. 3. 等等
            next_q = q_section.find(f"{count+1}. ")
            
            if(next_q == -1):
                target_dict[q_section[q+3:a_start]] = a_section[a_start+3:]     #substring last question / 最後一個問題的部分
                break
                
            a = a_section.find(f"{count}. ")
            next_a = a_section.find(f"{count+1}. ")
            
            target_dict[q_section[q+3:next_q]] = a_section[a+3:next_a]
            count += 1

        return target_dict
    
    
    def get_numbered_qa(section, target_dict):           #method 3
        count = 1
        while True:
            q = section.find(f"問題{count}：")        #find index of questions / 找數字：e.g. 1. 2. 3. 等等
            next_q = section.find(f"問題{count+1}：")

            a = section.find(f"答案{count}：")

            if(next_q == -1):
                target_dict[section[q+4:a]] = section[a+4:]     #substring last question / 最後一個問題的部分
                break

            target_dict[section[q+4:a]] = section[a+4:next_q+q]
            count += 1
            section = section[next_q:]

        return target_dict
    
    
    def which_method(section):         #decide which method should be used to substring qa / 判斷哪組問題應該用哪個function去跑
        if section.find('問題1：') != -1:
            return "method 3"
        
        if section.find('問題：') != -1:
            q1 = section.find('問題：')
            
            if section[q1+3:].find('1.') != -1:
                return "method 2"
            else:
                return "method 1"
            
        return "no method"

    
    def content_to_qa_1(unprocess_qa, section, temp):
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "基本用繁體中文，專業術語或引用其他人的話語時可以涵蓋其他語言的內容。所有的問題以'問題：'作為開頭，所有答案以'答案：'作為開頭。"},
                    {"role": "system", "content": "根據我所給你的內容產出涵蓋所有重要訊息的問題和他所對應的答案。"},
                    {"role": "assistant", "content": f"文章：{section}"}
                ],
                temperature = 0.2
            )
            #unprocess_newq: chatgpt直接輸出的答案，不做任何修改
            unprocess_qa.append(completion.choices[0].message['content'])
        except BaseException:
            unprocess_qa.append("")
        return unprocess_qa
    
    
    def get_q_from_unprocess(made_q_dict, unprocess_qa, ind):
        result = which_method(unprocess_qa[ind])
        if result == "no_result":
            return made_q_dict
        if result == "method 1":
            target_dict = {}
            made_q_dict[ind] = get_qa_1by1(unprocess_qa[ind], target_dict)
            return made_q_dict
        if result == "method 2":
            target_dict = {}
            made_q_dict[ind] = get_qa_by_list(unprocess_qa[ind], target_dict)
            return made_q_dict
        if result == "method 3":
            target_dict = {}
            made_q_dict[ind] = get_numbered_qa(unprocess_qa[ind], target_dict)
            return made_q_dict
    
    
    def content_to_qa_2(unprocess_qa, new_content, ind, temp):
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "基本用繁體中文，專業術語或引用其他人的話語時可以涵蓋其他語言的內容。所有的問題以'問題：'作為開頭，所有答案以'答案：'作為開頭。"},
                    {"role": "system", "content": "根據我所給你的內容產出涵蓋所有重要訊息的問題和他所對應的答案。"},
                    {"role": "assistant", "content": f"文章：{new_content[ind]}"}
                ],
                temperature = 0.6 
            )
            unprocess_qa[ind] = completion.choices[0].message['content']
        except BaseException:
            unprocess_qa[ind] = unprocess_qa[ind]
        return unprocess_qa
    
    
    
    #import all needed libraries
    import pdfplumber          #get content from pdf
    import pandas as pd        #form dataframe -> later form excel
    import os                  #libraries needed for openai api
    import openai
    openai.api_key = API_KEY #variable
    
    #get content from pdf
    pdf = pdfplumber.open(pdf_route) #variable
    content = []
    for parts in pdf.pages:
        content.append(parts.extract_text())
        
    #find all pages with QA sets / 找到含有Q&A的問題部分並提取出來
    if with_qa:
        withqa, content_q = withqa(content, question_symbol, answer_symbol)
    else:
        withqa = []
    
    #find all sections to rewrite / 找到含有list的問題並重寫    
    if with_list:
        to_article, withoutlist, need_rewrite, unprocess_article = withlist(content)
        new_content = form_new_content(content, to_article, withoutlist, withqa, need_rewrite, unprocess_article)
    else:
        new_content = form_new_content(content)
    
    #turn all verbal data into QA / 把new_content裡面所有內容轉成qa
    unprocess_qa = []
    temp = 0.2
    for section in new_content:
        unprocess_qa = content_to_qa_1(unprocess_qa, section, temp)
    
    
    #Turn all raw QA to correct data type / 把unprocess_qa里的問題們換成1Q1A的形式
    made_q_dict = {}
    run_again = [] 
    for ind in range(len(unprocess_qa)):
        made_q_dict = get_q_from_unprocess(made_q_dict, unprocess_qa, ind)
    run_again = [x for x in list(range(len(unprocess_qa)))  if x not in set(made_q_dict.keys())]  #raw QA that cannot be transfered to the required data type
    
    
    #If element in run_again exist, run previous step again. If elements in run_again exist after the second trial, the question would be selected for humans to determine
    #if如果有run_again就跑第二輪的unprocess_qa, 如果還有run_again就直接丟進human_look讓人類去判斷了
    temp += 0.4
    human_look = []
    if len(run_again) != 0:
        for ind in run_again:
            unprocess_qa = content_to_qa_2(unprocess_qa, new_content, ind, temp)
            made_q_dict = get_q_from_unprocess(made_q_dict, unprocess_qa, ind)
        
        run_again = [x for x in run_again if x not in set(made_q_dict.keys())]
        for x in run_again:
            human_look.append(unprocess_qa[x])
    
    #Save all 1Q1A to all_dict and output in dataframe / 最終把所有存進allq_dict然後以dataframe輸出
    allq_dict = {}       #Dictionary with all QA / 包含所有問題的dictionary
    for section_q in made_q_dict:           #All GPT generated QA / 把chatgpt生成的問題全部放到這裡
        for key, value in made_q_dict[section_q].items():
            allq_dict[key] = value
    if with_qa:
        for key, value in content_q.items():        #All existing QA in pdf / 把原文里就有的問題放到這裡
            allq_dict[key] = value
    allq_df = pd.DataFrame(list(allq_dict.items()))
    allq_df = allq_df.rename({0:'questions', 1:'answers'}, axis='columns')
    allq_df.loc[:, '人工查看列表'] = human_look + [""]*(len(allq_df.index) - len(human_look))
    
    pd.set_option('display.max_colwidth', None)
    
    file = allq_df.to_excel(excel_route)

    return file
    

In [3]:
Target_pdf = ''
ouput_excel = ''

file = API_func(API_KEY, Target_pdf, True, False, ouput_excel)
